.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u
from datetime import datetime, timedelta
from astropy.coordinates import SkyCoord, EarthLocation, AltAz, ICRS
from ctapipe.coordinates import CameraFrame
from lstchain.reco.utils import clip_alt
import sys, os, glob, re, tables
import pandas as pd
pd.set_option("display.max_columns", None) #

import utils

.

In [ ]:
run_number = 3599

source_name = "Crab"

# Use it in case you have all data in same subdirectory
root_data = "/fefs/aswg/workspace/juan.jimenez/data/cherenkov_transparency_corrections/crab/"

input_path_dl1 = None # Set to None if you want to use the standard one in IT cluster
input_path_dl1_scaled = f"{root_data}dl1_merged_scaled/dl1_LST-1.Run{run_number:05}.h5" #

input_path_dl3 = f"{root_data}dl3_total/dl3_LST-1.Run{run_number:05}.fits"
input_path_dl3_scaled = f"{root_data}dl3_scaled_total/dl3_LST-1.Run{run_number:05}.fits" #

input_path_dl2 = f"{root_data}dl2_radec/dl2_LST-1.Run{run_number:05}_radec.h5"
input_path_dl2_scaled = f"{root_data}dl2_scaled_radec/dl2_LST-1.Run{run_number:05}_radec.h5" #


.

In [ ]:
# Getting the DL1 filename for the file and the datacheck as well
path_dl1        = utils.find_dl1_fname(run_number) if input_path_dl1 == None else input_path_dl1
path_dl1_dcheck = utils.find_dl1_fname(run_number, dchecking=True, print_details=False)

path_dl2, path_dl2_radec, path_dl3 = input_path_dl2, input_path_dl2_radec, input_path_dl3
path_dl1_scaled, path_dl2_scaled = input_path_dl1_scaled, input_path_dl2_scaled             #
path_dl2_radec_scaled, path_dl3_scaled = input_path_dl2_radec_scaled, input_path_dl3_scaled #

table_dl1_datacheck, table_dl1, table_dl2, table_dl1_scaled, table_dl2_scaled = utils.open_files(global_variables=globals()) #

In [ ]:
# Reading DL1 and DL2 data

.

In [ ]:
# Reading datacheck data
# Direct quantities
srun_number = table_dl1_datacheck.cosmics.col("subrun_index")
srun_elapsed_time = table_dl1_datacheck.cosmics.col("elapsed_time")
srun_cumulative_time = np.cumsum(srun_elapsed_time)
srun_num_events = table_dl1_datacheck.cosmics.col("num_events")
srun_num_cleaned_events = table_dl1_datacheck.cosmics.col("num_cleaned_events")
srun_mean_az = np.rad2deg(table_dl1_datacheck.cosmics.col("mean_az_tel"))
srun_tel_ra  = table_dl1_datacheck.cosmics.col("tel_ra")
srun_tel_dec = table_dl1_datacheck.cosmics.col("tel_dec")

# Some manipulations
srun_dragon_time = np.array([f[0] for f in table_dl1_datacheck.cosmics.col("dragon_time")])
srun_charge_mean = np.array([np.mean(f) for f in table_dl1_datacheck.cosmics.col("charge_mean")])
srun_charge_stdv = np.array([np.mean(f) for f in table_dl1_datacheck.cosmics.col("charge_stddev")])
srun_mean_zd = 90 - np.rad2deg(table_dl1_datacheck.cosmics.col("mean_alt_tel"))
srun_num_non_cleaned_events = srun_num_events - srun_num_cleaned_events

# Calculations
srun_rate_events = srun_num_events / srun_elapsed_time
srun_rate_cleaned_events = srun_num_cleaned_events / srun_elapsed_time

# Histogram binnings
columns_binnings = table_dl1_datacheck.histogram_binning.colnames
columns_hist = [col for col in table_dl1_datacheck.cosmics.colnames if "hist_" in col]
for colname in columns_binnings:
    _bins_ = table_dl1_datacheck.histogram_binning.col(colname)[0]
    if len(_bins_.shape) == 1: # We get the binning if is simply an array (1-dimensional)
        globals()[colname.replace("hist", "bins")] = _bins_
        # Computing central bins, in order to do it, we check for logarithmic scales
        if np.sum(np.diff(np.diff(_bins_))) < 1e-14:
            globals()[colname.replace("hist", "bins") + "_center"] = (_bins_[1:] + _bins_[:-1]) / 2
        else:
            globals()[colname.replace("hist", "bins") + "_center"] = (_bins_[1:] * _bins_[:-1]) ** 0.5

for colname in columns_hist:
    counts = table_dl1_datacheck.cosmics.col(colname)
    if len(counts.shape) == 2: # We get the binning if is simply N arrays (1-dimensional)
        globals()[colname] = counts

In [ ]:
# Reading the cuts used from DL3


In [ ]:
hist_width[0]

In [ ]:
for col in columns_hist:
    print(col)
    bins = globals()[col.replace("hist", "bins")+"_center"]
    plt.title(col)
    plt.plot(bins, globals()[col][10])
    if np.sum(np.diff(np.diff(bins))) > 1e-14:
        plt.loglog()
    plt.show()
# plt.xlim(4, 200)

In [ ]:
fig, ax = plt.subplots(figsize=(7, 3))

ax.plot(srun_number, srun_elapsed_time, ds="steps-mid")

# ax.set_yscale("log")
ax.grid()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(7, 3))

ax.plot(srun_cumulative_time, srun_rates, ds="steps-post")
ax.plot(srun_cumulative_time, srun_rate_cleaned_events, ds="steps-post")

# ax.set_yscale("log")
ax.grid()
plt.show()

In [ ]:
file_dl1_datacheck.cosmics.col("charge_mean")

In [ ]:
file_dl1_datacheck.cosmics

In [ ]:
file_dl1_datacheck.cosmics.col("dragon_time")

In [ ]:
# Source data
source_coordinates = SkyCoord.from_name(source_name, frame="icrs")
source_ra, source_dec = source_coordinates.ra.deg, source_coordinates.dec.deg

In [ ]:
plt.plot(srun_tel_ra, srun_tel_dec, ".");

In [ ]:
file_dl1_datacheck.dl1datacheck.cosmics.col("num_ucts_jumps")

In [ ]:
pd.read_hdf(path_dl1_dcheck, "/dl1datacheck/cosmics")

In [ ]:
tab = pd.read_hdf(path_dl1_scaled, "/dl1/event/telescope/parameters/LST_LSTCam")

In [ ]:
tab

In [ ]:
find_run_datacheck(run_number)

In [ ]:
path_dl1_datacheck = 